# Function

In [3]:
!pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display, Audio
from google.colab import output
import numpy as np
import io
import base64

# Create start and stop buttons
start_button = widgets.Button(description="Start Recording")
stop_button = widgets.Button(description="Stop Recording")

recording = False

# Function to start recording
def start_recording(b):
    global recording
    recording = True
    output.eval_js('startRecording()')
    print("Recording started...")

# Function to stop recording and save the file
def stop_recording(b):
    global recording
    if not recording:
        print("No recording in progress.")
        return

    recording = False
    audio_data = output.eval_js('stopRecording()')

    if audio_data is None:
        print("No audio data recorded.")
        return

    # Decode base64 data
    audio_bytes = base64.b64decode(audio_data['audio'])

    # Create and display Audio widget
    audio_obj = Audio(data=audio_bytes, rate=audio_data['sampleRate'])
    display(audio_obj)

    # Save as WAV file
    with open("recorded_audio.wav", "wb") as f:
        f.write(audio_bytes)

    print("Audio saved as 'recorded_audio.wav'")

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [12]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [5]:
#from google.colab import output
#output.disable_custom_widget_manager()

In [6]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-4fs0efcs
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-4fs0efcs
  Resolved https://github.com/openai/whisper.git to commit 25639fc17ddc013d56c594bfbf7644f2185fad84
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803321 sha256=c498e0b932c8d5867f33cf3e77ceb37889e386e2cf5b1d7ca3123f982b5961f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-xd65fftf/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [7]:
!apt update && apt install -y ffmpeg

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,159 kB]
Get:14 http://security.

In [8]:
import whisper

# Load Whisper model (we'll use "base" here for demo purposes)
model = whisper.load_model("base")

# Transcribe function that detects the language first
def detect_and_transcribe(audio_file):
    # Load the audio file and detect the language
    audio = whisper.load_audio(audio_file)
    audio = whisper.pad_or_trim(audio)  # Ensure it's the right length for processing

    # Get the mel spectrogram
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # Detect the language
    _, probs = model.detect_language(mel)
    detected_language = max(probs, key=probs.get)

    print(f"Detected Language: {detected_language}")

    # Transcribe the audio file
    result = model.transcribe(audio_file)

    # Print transcription
    print(f"Transcription ({detected_language}): {result['text']}")

    return result



100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 55.1MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [9]:
import pandas as pd
from datetime import datetime

# Function to store transcription data into a pandas DataFrame
def store_transcription_in_df(transcription_data, dataframe=None):
    # Extract the necessary fields from the transcription data
    text = transcription_data['text']
    segments = transcription_data['segments']
    language = transcription_data['language']

    # Get the current date and time
    date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Prepare a list to store each segment's data
    data = []

    for segment in segments:
        data.append({
            "date": date,
            "language": language,
            "start_time": segment['start'],
            "end_time": segment['end'],
            "segment_text": segment['text'],
            "avg_logprob": segment['avg_logprob'],
            "compression_ratio": segment['compression_ratio'],
            "no_speech_prob": segment['no_speech_prob']
        })

    # Create a Pandas DataFrame from the data
    new_df = pd.DataFrame(data)

    # If no dataframe is provided, return the new DataFrame
    if dataframe is None:
        print(new_df)
        return new_df

    # If a dataframe is provided, append the new data and return the updated DataFrame
    updated_df = pd.concat([dataframe, new_df], ignore_index=True)

    print(updated_df)  # Show the updated DataFrame
    return updated_df

# Example usage with a transcription (pass the transcription result from Whisper)
transcription_data = {
    'text': ' Hello my dream was about a lebricon that was running around on a rainbow hill',
    'segments': [
        {
            'id': 0,
            'seek': 0,
            'start': 0.0,
            'end': 11.0,
            'text': ' Hello my dream was about a lebricon that was running around on a rainbow hill',
            'tokens': [50364, 2425, 452, 3055, 390, 466, 257, 476, 65, 1341, 266, 300, 390, 2614, 926, 322, 257, 18526, 10997, 50914],
            'temperature': 0.0,
            'avg_logprob': -0.43285406203497023,
            'compression_ratio': 1.0845070422535212,
            'no_speech_prob': 0.06515374034643173
        }
    ],
    'language': 'en'
}

# Create a new DataFrame with the transcription
df = store_transcription_in_df(transcription_data)

# Simulate appending new transcription data to the existing DataFrame
new_transcription_data = {
    'text': ' Last night I dreamt of flying.',
    'segments': [
        {
            'id': 1,
            'seek': 0,
            'start': 0.0,
            'end': 8.0,
            'text': ' Last night I dreamt of flying.',
            'tokens': [1234, 5678, 91011],
            'temperature': 0.0,
            'avg_logprob': -0.123456,
            'compression_ratio': 1.2,
            'no_speech_prob': 0.1
        }
    ],
    'language': 'en'
}

                  date language  start_time  end_time  \
0  2024-10-07 13:32:47       en         0.0      11.0   

                                        segment_text  avg_logprob  \
0   Hello my dream was about a lebricon that was ...    -0.432854   

   compression_ratio  no_speech_prob  
0           1.084507        0.065154  


In [10]:
import openai

openai.api_key = "YOUR_OPENAI_API_KEY"

def clean_transcription(text):
    prompt = f"Please clean up this dream transcription and make it more readable: {text}"
    response = openai.Completion.create(
        engine="gpt-4",  # Or gpt-3.5-turbo
        prompt=prompt,
        max_tokens=500
    )
    return response.choices[0].text.strip()

cleaned_dream = clean_transcription(dream_text)
print("Cleaned Dream:", cleaned_dream)

ModuleNotFoundError: No module named 'openai'

In [ ]:
def generate_title(text):
    prompt = f"Generate a creative and fitting title for this dream: {text}"
    response = openai.Completion.create(
        engine="gpt-4",
        prompt=prompt,
        max_tokens=20  # Limit title length
    )
    return response.choices[0].text.strip()

dream_title = generate_title(cleaned_dream)
print("Dream Title:", dream_title)

# AI Dream Journal

Make recording

In [13]:
#record invoice


# Connect buttons to functions
start_button.on_click(start_recording)
stop_button.on_click(stop_recording)

# Display buttons
display(start_button, stop_button)

# JavaScript to handle audio recording
output.eval_js('''
    let recorder, stream;

    async function startRecording() {
        stream = await navigator.mediaDevices.getUserMedia({ audio: true });
        recorder = new MediaRecorder(stream);
        chunks = [];
        recorder.ondataavailable = e => chunks.push(e.data);
        recorder.start();
    }

    function stopRecording() {
        return new Promise(resolve => {
            recorder.onstop = () => {
                const blob = new Blob(chunks, { type: 'audio/wav' });
                chunks = [];
                stream.getTracks().forEach(track => track.stop());
                const reader = new FileReader();
                reader.onload = () => {
                    const base64data = reader.result.split(',')[1];
                    resolve({audio: base64data, sampleRate: 44100});
                };
                reader.readAsDataURL(blob);
            };
            recorder.stop();
        });
    }
''')

Button(description='Start Recording', style=ButtonStyle())

Button(description='Stop Recording', style=ButtonStyle())

Recording started...


Audio saved as 'recorded_audio.wav'
Recording started...


Audio saved as 'recorded_audio.wav'


 Detect language and transcribe voice recording

In [16]:
# Example: Replace 'your_audio_file.wav' with your audio file
transcription = detect_and_transcribe("/content/recorded_audio.wav")

Detected Language: en


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription (en):  Hello, this is a test for a dream.


In [17]:
print(transcription)

{'text': ' Hello, this is a test for a dream.', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 7.0, 'text': ' Hello, this is a test for a dream.', 'tokens': [50364, 2425, 11, 341, 307, 257, 1500, 337, 257, 3055, 13, 50714], 'temperature': 0.0, 'avg_logprob': -0.6019412921025202, 'compression_ratio': 0.8717948717948718, 'no_speech_prob': 0.05958086624741554}], 'language': 'en'}


Store transcription in a new DataFrame

In [ ]:
# Append the new transcription data to the existing DataFrame or create new dataframe
df = store_transcription_in_df(transcription)
df.head()

                  date language  start_time  end_time  \
0  2024-10-03 14:52:46       en         0.0       6.0   

                  segment_text  avg_logprob  compression_ratio  no_speech_prob  
0   Hello, this is a new test.    -0.658347             0.8125         0.03686  


In [1]:
#column_headers = list(df.columns.values)
#print("The Column Header :", column_headers)

Load your Dream journal / Load csv into Dataframe

In [18]:
# Load csv to Dataframe
# If the filename is 'dream_journal.csv', you can load it like this:
df_dreamjournal = pd.read_csv('dream_journal.csv')

Add transcription to your dream journal / add transcription to Dataframe

In [19]:
# Append the new transcription data to the existing DataFrame or create new dataframe
df = store_transcription_in_df(transcription, df_dreamjournal)

                  date language  start_time  end_time  \
0  2024-10-03 14:52:46       en         0.0       6.0   
1  2024-10-07 13:36:52       en         0.0       7.0   

                          segment_text  avg_logprob  compression_ratio  \
0           Hello, this is a new test.    -0.658347           0.812500   
1   Hello, this is a test for a dream.    -0.601941           0.871795   

   no_speech_prob  
0        0.036860  
1        0.059581  


In [20]:
df.head()

,date,language,start_time,end_time,segment_text,avg_logprob,compression_ratio,no_speech_prob
0,2024-10-03 14:52:46,en,0.0,6.0,"Hello, this is a new test.",-0.658347,0.812500,0.036860
1,2024-10-07 13:36:52,en,0.0,7.0,"Hello, this is a test for a dream.",-0.601941,0.871795,0.059581


Save changes to dream journal Dataframe and download it

In [21]:
#save df
# Save the DataFrame as a CSV file
df.to_csv('dream_journal.csv', index=False)  # index=False to not include the DataFrame index

In [22]:
from google.colab import files

# Download the CSV file
files.download('dream_journal.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>